# Transfer Learning

### Transfer learning is the process of taking a pretrained Neural Network and using it for our own use.
In this process, we are loading a pretrained neural network, removing the flatten and output layers so we'll have a network that is trained only to extract features from image.

Then, we are adding layers to it for our use - like falttening and output according to our dataset - and then we train the model on our dataset.

This way, the pretrained network performs the complex task of feature extraction and we are adding the simple task of labeling

In [69]:
import numpy as np
from pathlib import Path
import os
import joblib
from keras.preprocessing import image
from keras.applications import vgg16
import keras
from keras.models import Sequential,model_from_json
from keras.layers import Dense,Dropout,Flatten

base_path = "datasets"
dog_path = Path(os.path.join(base_path,"dogs"))
not_dog_path = Path(os.path.join(base_path,"not_dogs"))

In [70]:
images = []
labels = []
# This function addes data to images and labels arrays
# path - a path for a Path object directory
# label - the number represents the label for the dir
def add_data_to_arrays(path,label):
    # for each image in the path
    for img in path.glob("*.png"):
        # load the image to an array
        img = keras.utils.load_img(img)
        img_array = image.image_utils.img_to_array(img=img)
        # append the image and the labels to the arrays
        images.append(img_array)
        labels.append(label)
add_data_to_arrays(not_dog_path,0)
add_data_to_arrays(dog_path,1)

In [71]:
# convert the data to numpy arrays
x_train = np.array(images)
y_train = np.array(labels)
# perform preprocessing on the x data
x_train = vgg16.preprocess_input(x_train)


In [72]:
# This loads a pretrained vgg16 CNN structure with the weights of a network trained by imagenet dataset
# The include_top=False means that we don't want to include the flattening output layer, because in transfer learning we are using the CNN only for feature extraction
pretrained_nn = vgg16.VGG16(weights="imagenet",include_top=False,input_shape=(64,64,3))


In [73]:
# Extract features of each image in one pass - the predict this time finishes in the last Convolutional layer so we'll get only the features of each image
features_x = pretrained_nn.predict(x_train)
# Dumping the results in data files
joblib.dump(features_x,"x_train.dat")
joblib.dump(y_train,"y_train.dat")

2/2 [==============================] - 1s 210ms/step


['y_train.dat']

In [74]:
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

### The model

Since we are using transfer learning, once we used the pretrained model for creating data, we'll use another model to label this data

In [75]:
model = Sequential()

model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [76]:
model.fit(x=x_train,y=y_train,epochs=10,shuffle=True)

Epoch 1/10
2/2 [==============================] - 0s 5ms/step - loss: 12.2340 - accuracy: 0.4655
Epoch 2/10
2/2 [==============================] - 0s 4ms/step - loss: 2.0361 - accuracy: 0.9138
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 2.2122e-09 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 5ms/step - loss: 0.2585 - accuracy: 0.9828
Epoch 5/10
2/2 [==============================] - 0s 4ms/step - loss: 0.0327 - accuracy: 0.9828
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 0.3547 - accuracy: 0.9828
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.3005 - accuracy: 0.9828
Epoch 8/10
2/2 [==============================] - 0s 4ms/step - loss: 1.7064e-08 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 5ms/step - loss: 0.1733 - accuracy: 0.9828
Epoch 10/10
2/2 [==============================] - 0s 5ms/step - loss: 1.2785e-13 - accuracy: 1.0000


In [77]:
model_structure = model.to_json()
f = Path("model_structure_2.json")
f.write_text(model_structure)
model.save_weights("model_weights_2.h5")

In [78]:
f = Path("model_structure_2.json")
model_structure = f.read_text()
model = model_from_json(model_structure)

model.load_weights("model_weights_2.h5")

img = keras.utils.load_img("dog.png",target_size=(64,64))
img_array = image.image_utils.img_to_array(img)
images = np.expand_dims(img_array,axis=0)
images = vgg16.preprocess_input(images)
print(images.shape)

(1, 64, 64, 3)


In [83]:
# feature_extraction_model = vgg16.VGG16(weights="imagenet",include_top=False,input_shape=(64,64,3))

features = pretrained_nn.predict(images)

results = model.predict(features)

result_likelihood = results[0][0]

print(result_likelihood)


1/1 [==============================] - 0s 17ms/step
1.0
